In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset,random_split
import torch.nn.functional as F
import time
import os
import re
from itertools import chain
from transformers import BertTokenizer, BertForMaskedLM
from IPython.display import clear_output
import pandas as pd

PRETRAINED_MODEL_NAME = "bert-base-chinese" #英文pretrain(不區分大小寫)
print(torch.__version__)

/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu-gpu/anaconda3/e

1.1.0


In [4]:
# get pre-train tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
vocab = tokenizer.vocab
print("dict size", len(vocab))

# see some token and index mapping
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids): #隨便看幾個字
    print("{0:15}{1:10}".format(t, id))

dict size 21128
token               index          
-------------------------
昱                    3222
廃                    2440
##骄                 20791
萊                    5844
はすへて                11899
雄                    7413
##虢                 19056
##鲛                 20887
玩                    4381
##紉                 18206


In [5]:
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')
total = pd.concat([train, val])
total = total.reset_index(drop=True)
total_0 = total[total["Label"] == 0]
total_1 = total[total["Label"] == 1]

In [6]:
# Label_1
num_1 = total_1.shape[0]
random_1 = random.sample(range(num_1), num_1)
train_1 = total_1.iloc[random_1[:108]]
test_1 = total_1.iloc[random_1[108:135]]
# Label_0
num_0 = total_0.shape[0]
random_0 = random.sample(range(num_0), num_0)
train_0 = total_0.iloc[random_0[:400]]
test_0 = total_0.iloc[random_0[400:500]]
# train_val_1
train_val_1 = train_1.shape[0]
random_train_val = random.sample(range(train_val_1), train_val_1)
train__1 = train_1.iloc[random_train_val[:97]]
val_1 = train_1.iloc[random_train_val[97:108]]
# train_val_0
train_val_0 = train_0.shape[0]
random_train_val = random.sample(range(train_val_0), train_val_0)
train__0 = train_0.iloc[random_train_val[:360]]
val_0 = train_0.iloc[random_train_val[360:400]]
# train_val
train = pd.concat([train__1, train__0])
train = train.reset_index(drop=True)
val = pd.concat([val_1, val_0])
val = val.reset_index(drop=True)
# test
test = pd.concat([test_1, test_0])
test = test.reset_index(drop=True)

In [7]:
def toList(data):
    df = []
    for j in range(len(data)):
        index = []
        index.append(data.iloc[j]['Content'])
        index.append(data.iloc[j]['Label'])
        df.append(index)
    return df

In [26]:
train = toList(train)
val = toList(val)
test = toList(test)
test

[['一直想自殺 但都找不到可以100%自殺的方法······自殺只要不成功 就只會讓現在的狀況更加惡化······舉凡：癱瘓、植物人等 大概只剩無條件的安樂死 但是好像沒有國家可以合法這麼做······????為什麼連想脫離痛苦',
  1],
 ['在北上的路途中看見了防治自殺的文章，我就不自覺的哭出來了，我討厭我的家庭，我愛我的家庭，我好想走掉，我不敢走掉，醫生感覺只想賺我的藥錢而已，xxx, 你要我怎麼辦要我怎麼辦',
  1],
 ['六月初的時候，我發現男友肉體出軌，我看見他跟女同事的line對話。我控制穩定一年的躁鬱爆發，我每天都想死，可是我離不開他。七月中終於去看醫生，然後躁症發作，我開始每天都心情很好，也睡很少。七月底正式分手，我回老家一陣子，雖然偶爾會難過可是還過得下去，我很努力在過新生活。八月初我回家了，有一天早上我被強暴了，前男友說了很多我不堪的過去，結束後我完全崩潰了，然後就只剩下麻木。我看的兩個醫生都叫我去住院，但我的家人說不要去。我跟我的好朋友變炮友，我知道自己已經壞了死了，我只剩這個方式活著。我每天都在問自己為什麼要活著。為什麼我還活著？--',
  1],
 ['吃了很多顆藥卻還是睡不著好累，但睡不著起來拼了幾塊拼圖但因為頭暈決定躺回去但還是睡不著好累好累什麼都不想做了吵個架真的傷神可不可以不要逼我了呢可不可以多吃很多很多藥然後再也不要醒來--',
  1],
 ['我之前長期吃藥穩定了一年 但因為吃很重思樂康變很胖 最近問醫生說思樂康可不可以減 他就說先減50mg (原本吃400mg) 看起來好像沒甚麼 但我減到現在也才半個禮拜 腦內一直有聲音叫我去死 腦部活動大幅增加 一直想發文 一直想自殺 很煩 差50mg差那麼多 是不是又得加回來 我就一輩子過胖 一輩子不用再創作 (因為腦部活動減少會沒有靈感) 這樣活一輩子有甚麼意義可言 到底--',
  1],
 ['一直在碰到弄髒的疑慮中洗不停的手跟全身記不得每天要用掉多少肥皂跟沐浴乳洗髮乳每天衣服也是一直換一直換一直洗下雨的時候衣服乾不了很痛苦洗到好厭世好累好難過一直邊洗邊哭分不清到底是強迫讓我憂鬱還是憂鬱合併強迫了已經提了離職做到五月底也已經看醫生改藥了一點進展也沒有反而吃了藥之後坐立不安只要醒著不管躺著趴著坐著整個人都很難受是否只能結束自己才能停止這無盡的動作好想要一刀往自己喉嚨

In [27]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df #its list [['text1',label],['text2',label],...]
        self.len = len(self.df)
        self.maxlen = 250 #限制文章長度(若你記憶體夠多也可以不限)
        self.tokenizer = tokenizer  # we will use BERT tokenizer
        
    def __getitem__(self, idx):
        origin_text = self.df[idx][0]
        text_a = self.df[idx][0]
        text_b = None  #for natural language inference
        #label_tensor = None #in our case, we have label
        label_id = self.df[idx][1]
        label_tensor = torch.tensor(label_id)
        
        # 建立第一個句子的 BERT tokens
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a[:self.maxlen] + ["[SEP]"]
        len_a = len(word_pieces)
        
        if text_b is not None:
            tokens_b = self.tokenizer.tokenize(text_b)
            word_pieces += tokens_b + ["[SEP]"]
            len_b = len(word_pieces) - len_a
            
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        if text_b is None:
            segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
        elif text_b is not None:
            segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
            
        return (tokens_tensor, segments_tensor, label_tensor, origin_text)
    
    def __len__(self):
        return self.len

In [28]:
trainset = MyDataset(train, tokenizer=tokenizer)
valset = MyDataset(val, tokenizer=tokenizer)
testset = MyDataset(test, tokenizer=tokenizer)

In [145]:
# def MyDataset(Dataset):
#     data_len = len(Dataset)
#     maxlen = 250
#     text_a = Dataset['Content']
#     origin_text = Dataset['Content']
#     text_b = None  #for natural language inference
#     label_id = Dataset['Label']
#     label_tensor = torch.tensor(label_id)

#     # 建立第一個句子的 BERT tokens
#     word_pieces = []
#     for i in range(len(text_a)):
#         word_pieces += ["[CLS]"]
#         tokens_a = tokenizer.tokenize(text_a[i])
#         word_pieces += tokens_a[:maxlen] + ["[SEP]"]
#         len_a = len(word_pieces)
#         if text_b is not None:
#             tokens_b = self.tokenizer.tokenize(text_b)
#             word_pieces += tokens_b + ["[SEP]"]
#             len_b = len(word_pieces) - len_a
            
#     # 將整個 token 序列轉換成索引序列
#     ids = tokenizer.convert_tokens_to_ids(word_pieces)
#     tokens_tensor = torch.tensor(ids)
    
#     # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
#     if text_b is None:
#         segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
#     elif text_b is not None:
#         segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)

#     # 將 tokens_tensor 還原成文本
#     tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
    
#     print('token:\n',tokens,'\n')
#     print('origin_text:\n',origin_text,'\n')
#     print('tokens_tensor:\n',tokens_tensor,'\n')
#     print('segment tensor:\n',segments_tensor)
#     return (tokens_tensor, segments_tensor, label_tensor, origin_text, tokens)

In [29]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
""""
create_mini_batch(samples)吃上面定義的mydataset
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
#collate_fn: 如何將多個樣本的資料連成一個batch丟進 model
#截長補短後要限制attention只注意非pad 的部分
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
        
    # zero pad到該batch下最長的長度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

# 初始化一個每次回傳 batch size 個訓練樣本的 DataLoader
# 利用 'collate_fn' 將 list of samples 合併成一個 mini-batch
BATCH_SIZE = 91
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)

data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data
print(tokens_tensors)
print(segments_tensors)
print(masks_tensors)
print(label_ids)

tensor([[ 101,  791, 1921,  ...,    0,    0,    0],
        [ 101, 1348, 1526,  ..., 1962, 7432,  102],
        [ 101, 1184, 7369,  ...,    0,    0,    0],
        ...,
        [ 101, 2682, 6206,  ...,    0,    0,    0],
        [ 101, 3315, 3189,  ..., 2533, 7768,  102],
        [ 101, 3219, 3241,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0

In [30]:
from transformers import BertForSequenceClassification

NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)


print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:10} {}".format(name, module))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout    Dropout(p=0.1)
classifier Linear(in_features=768, out_features=2, bias=True)


In [34]:
%%time
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 10

for epoch in range(EPOCHS):
    correct = 0
    #total = 0
    train_loss , val_loss = 0.0 , 0.0
    train_acc, val_acc = 0, 0
    n, m = 0, 0
    model.train()
    for data in trainloader:
        n += 1
        tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        # outputs 的順序是 "(loss), logits, (hidden_states), (attentions)"
        
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        
        #get prediction and calulate acc
        logits = outputs[1]
        _, pred = torch.max(logits.data, 1)
        train_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())

        # 紀錄當前 batch loss
        train_loss += loss.item()
    
    #validation
    with torch.no_grad():
        model.eval()
        for data in valloader:
            m += 1
            tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]
            val_outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
            
            logits = val_outputs[1]
            _, pred = torch.max(logits.data, 1)
            val_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())
            val_loss += val_outputs[0].item()

    print('[epoch %d] loss: %.4f, acc: %.4f, val loss: %4f, val acc: %4f' %
          (epoch+1, train_loss/n, train_acc/n, val_loss/m,  val_acc/m  ))

print('Done')

device: cuda:0


RuntimeError: cublas runtime error : the GPU program failed to execute at /opt/conda/conda-bld/pytorch_1556653215914/work/aten/src/THC/THCBlas.cu:259

In [33]:
"cuda:1" if torch.cuda.is_available() else "cpu"

'cuda:1'